In [ ]:
import pandas as pd
import boto3
import os

In [ ]:
access_key_id = ""
access_secret_id = ""
bucket = ""

file_key_prefix = f"folders"
files = []
s3 = boto3.client("s3", aws_access_key_id=access_key_id, aws_secret_access_key=access_secret_id)

response = s3.list_objects_v2(Bucket=bucket, Prefix=file_key_prefix)

for content in response.get("Contents", []):
    s3_object = content["Key"]
    if s3_object.split("/")[-1] == "":
        continue
    else:
        files.append(s3_object)


def download_from_s3(file_key):
    temp_folder = "/folders"
    is_downloaded = []
    download_paths = []

    for f in file_key:
        fname = f.split("/")[-1]
        temp_path = os.path.join(temp_folder, fname)
        is_downloaded.append(os.path.exists(temp_path))
        download_paths.append(temp_path)

    is_downloaded_all = True
    for b in is_downloaded:
        is_downloaded_all = is_downloaded_all and b

    if not is_downloaded_all:
        client = boto3.client(
            "s3",
            aws_access_key_id=access_key_id,
            aws_secret_access_key=access_secret_id,
            use_ssl=True,
            endpoint_url=None,
        )

        for i, f in enumerate(file_key):
            if not is_downloaded[i]:
                print(is_downloaded[i])
                client.download_file(bucket, f, download_paths[i])

    return download_paths

downloaded_files = download_from_s3(files)
downloaded_files

In [ ]:
import pandas as pd
import json

# Initialize an empty list to store JSON objects
data_list = []
# Specify the path to the JSON file
for file in downloaded_files:
    # Read JSON objects from the file
    with open(file, "r") as json_file:
        for line in json_file:
            data = json.loads(line)
            data_list.append(data)

# Create a Pandas DataFrame from the list of JSON objects
df = pd.DataFrame(data_list)
df["total_spent_time"] = df["total_spent_time"].fillna(0).astype(int)
df